In [91]:
import numpy as np
import librosa
import math
import re
import os
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical 
from sklearn.model_selection import train_test_split

In [92]:
class AudioDataGenerator:
    
    
    def count_files(self):
        counter = 0
        for directory in os.listdir("./dataset3/audio"):
            for file in os.listdir("./dataset3/audio/" + directory):
                counter = counter + 1
                
        print(counter) # 64726 #d3 33653
    
    def flow_from_directory(self,dir_path):
        labels = []
        data = []
        counter = 0
        timeseries_length = 16
        hop_length = 512
        data = np.zeros((33653, timeseries_length, 33), dtype=np.float64)
      
        for directory in os.listdir(dir_path):
            category = directory
            for file in os.listdir(dir_path +"/"+category):
                y, sr = librosa.load(os.getcwd() + "\\dataset3\\audio\\" + category +"\\" + file)
                
                mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length= 512, n_mfcc= 33)
                data[counter, :, 0:33] = mfcc.T[0:timeseries_length, :]
            
                counter = counter + 1
                
                labels.append(category)
                
                print("File: {} extracted. Category {}".format(counter, category))
        
        return data, np.expand_dims(np.asarray(labels), axis=1)
       
            
               
                
               
        
        

In [93]:
generator = AudioDataGenerator()
#generator.count_files()
X_train, y_train = generator.flow_from_directory("./dataset3/audio")

File: 1 extracted. Category cat
File: 2 extracted. Category cat
File: 3 extracted. Category cat
File: 4 extracted. Category cat
File: 5 extracted. Category cat
File: 6 extracted. Category cat
File: 7 extracted. Category cat
File: 8 extracted. Category cat
File: 9 extracted. Category cat
File: 10 extracted. Category cat
File: 11 extracted. Category cat
File: 12 extracted. Category cat
File: 13 extracted. Category cat
File: 14 extracted. Category cat
File: 15 extracted. Category cat
File: 16 extracted. Category cat
File: 17 extracted. Category cat
File: 18 extracted. Category cat
File: 19 extracted. Category cat
File: 20 extracted. Category cat
File: 21 extracted. Category cat
File: 22 extracted. Category cat
File: 23 extracted. Category cat
File: 24 extracted. Category cat
File: 25 extracted. Category cat
File: 26 extracted. Category cat
File: 27 extracted. Category cat
File: 28 extracted. Category cat
File: 29 extracted. Category cat
File: 30 extracted. Category cat
File: 31 extracted.

File: 251 extracted. Category cat
File: 252 extracted. Category cat
File: 253 extracted. Category cat
File: 254 extracted. Category cat
File: 255 extracted. Category cat
File: 256 extracted. Category cat
File: 257 extracted. Category cat
File: 258 extracted. Category cat
File: 259 extracted. Category cat
File: 260 extracted. Category cat
File: 261 extracted. Category cat
File: 262 extracted. Category cat
File: 263 extracted. Category cat
File: 264 extracted. Category cat
File: 265 extracted. Category cat
File: 266 extracted. Category cat
File: 267 extracted. Category cat
File: 268 extracted. Category cat
File: 269 extracted. Category cat
File: 270 extracted. Category cat
File: 271 extracted. Category cat
File: 272 extracted. Category cat
File: 273 extracted. Category cat
File: 274 extracted. Category cat
File: 275 extracted. Category cat
File: 276 extracted. Category cat
File: 277 extracted. Category cat
File: 278 extracted. Category cat
File: 279 extracted. Category cat
File: 280 extr

File: 493 extracted. Category cat
File: 494 extracted. Category cat
File: 495 extracted. Category cat
File: 496 extracted. Category cat
File: 497 extracted. Category cat
File: 498 extracted. Category cat
File: 499 extracted. Category cat
File: 500 extracted. Category cat
File: 501 extracted. Category cat
File: 502 extracted. Category cat
File: 503 extracted. Category cat
File: 504 extracted. Category cat
File: 505 extracted. Category cat
File: 506 extracted. Category cat
File: 507 extracted. Category cat
File: 508 extracted. Category cat
File: 509 extracted. Category cat
File: 510 extracted. Category cat
File: 511 extracted. Category cat
File: 512 extracted. Category cat
File: 513 extracted. Category cat
File: 514 extracted. Category cat
File: 515 extracted. Category cat
File: 516 extracted. Category cat
File: 517 extracted. Category cat
File: 518 extracted. Category cat
File: 519 extracted. Category cat
File: 520 extracted. Category cat
File: 521 extracted. Category cat
File: 522 extr

File: 738 extracted. Category cat
File: 739 extracted. Category cat
File: 740 extracted. Category cat
File: 741 extracted. Category cat
File: 742 extracted. Category cat
File: 743 extracted. Category cat
File: 744 extracted. Category cat
File: 745 extracted. Category cat
File: 746 extracted. Category cat
File: 747 extracted. Category cat
File: 748 extracted. Category cat
File: 749 extracted. Category cat
File: 750 extracted. Category cat
File: 751 extracted. Category cat
File: 752 extracted. Category cat
File: 753 extracted. Category cat
File: 754 extracted. Category cat
File: 755 extracted. Category cat
File: 756 extracted. Category cat
File: 757 extracted. Category cat
File: 758 extracted. Category cat
File: 759 extracted. Category cat
File: 760 extracted. Category cat
File: 761 extracted. Category cat
File: 762 extracted. Category cat
File: 763 extracted. Category cat
File: 764 extracted. Category cat
File: 765 extracted. Category cat
File: 766 extracted. Category cat
File: 767 extr

File: 981 extracted. Category cat
File: 982 extracted. Category cat
File: 983 extracted. Category cat
File: 984 extracted. Category cat
File: 985 extracted. Category cat
File: 986 extracted. Category cat
File: 987 extracted. Category cat
File: 988 extracted. Category cat
File: 989 extracted. Category cat
File: 990 extracted. Category cat
File: 991 extracted. Category cat
File: 992 extracted. Category cat
File: 993 extracted. Category cat
File: 994 extracted. Category cat
File: 995 extracted. Category cat
File: 996 extracted. Category cat
File: 997 extracted. Category cat
File: 998 extracted. Category cat
File: 999 extracted. Category cat
File: 1000 extracted. Category cat
File: 1001 extracted. Category cat
File: 1002 extracted. Category cat
File: 1003 extracted. Category cat
File: 1004 extracted. Category cat
File: 1005 extracted. Category cat
File: 1006 extracted. Category cat
File: 1007 extracted. Category cat
File: 1008 extracted. Category cat
File: 1009 extracted. Category cat
File

...

File: 32729 extracted. Category zero
File: 32730 extracted. Category zero
File: 32731 extracted. Category zero
File: 32732 extracted. Category zero
File: 32733 extracted. Category zero
File: 32734 extracted. Category zero
File: 32735 extracted. Category zero
File: 32736 extracted. Category zero
File: 32737 extracted. Category zero
File: 32738 extracted. Category zero
File: 32739 extracted. Category zero
File: 32740 extracted. Category zero
File: 32741 extracted. Category zero
File: 32742 extracted. Category zero
File: 32743 extracted. Category zero
File: 32744 extracted. Category zero
File: 32745 extracted. Category zero
File: 32746 extracted. Category zero
File: 32747 extracted. Category zero
File: 32748 extracted. Category zero
File: 32749 extracted. Category zero
File: 32750 extracted. Category zero
File: 32751 extracted. Category zero
File: 32752 extracted. Category zero
File: 32753 extracted. Category zero
File: 32754 extracted. Category zero
File: 32755 extracted. Category zero
F

File: 32952 extracted. Category zero
File: 32953 extracted. Category zero
File: 32954 extracted. Category zero
File: 32955 extracted. Category zero
File: 32956 extracted. Category zero
File: 32957 extracted. Category zero
File: 32958 extracted. Category zero
File: 32959 extracted. Category zero
File: 32960 extracted. Category zero
File: 32961 extracted. Category zero
File: 32962 extracted. Category zero
File: 32963 extracted. Category zero
File: 32964 extracted. Category zero
File: 32965 extracted. Category zero
File: 32966 extracted. Category zero
File: 32967 extracted. Category zero
File: 32968 extracted. Category zero
File: 32969 extracted. Category zero
File: 32970 extracted. Category zero
File: 32971 extracted. Category zero
File: 32972 extracted. Category zero
File: 32973 extracted. Category zero
File: 32974 extracted. Category zero
File: 32975 extracted. Category zero
File: 32976 extracted. Category zero
File: 32977 extracted. Category zero
File: 32978 extracted. Category zero
F

File: 33176 extracted. Category zero
File: 33177 extracted. Category zero
File: 33178 extracted. Category zero
File: 33179 extracted. Category zero
File: 33180 extracted. Category zero
File: 33181 extracted. Category zero
File: 33182 extracted. Category zero
File: 33183 extracted. Category zero
File: 33184 extracted. Category zero
File: 33185 extracted. Category zero
File: 33186 extracted. Category zero
File: 33187 extracted. Category zero
File: 33188 extracted. Category zero
File: 33189 extracted. Category zero
File: 33190 extracted. Category zero
File: 33191 extracted. Category zero
File: 33192 extracted. Category zero
File: 33193 extracted. Category zero
File: 33194 extracted. Category zero
File: 33195 extracted. Category zero
File: 33196 extracted. Category zero
File: 33197 extracted. Category zero
File: 33198 extracted. Category zero
File: 33199 extracted. Category zero
File: 33200 extracted. Category zero
File: 33201 extracted. Category zero
File: 33202 extracted. Category zero
F

File: 33400 extracted. Category zero
File: 33401 extracted. Category zero
File: 33402 extracted. Category zero
File: 33403 extracted. Category zero
File: 33404 extracted. Category zero
File: 33405 extracted. Category zero
File: 33406 extracted. Category zero
File: 33407 extracted. Category zero
File: 33408 extracted. Category zero
File: 33409 extracted. Category zero
File: 33410 extracted. Category zero
File: 33411 extracted. Category zero
File: 33412 extracted. Category zero
File: 33413 extracted. Category zero
File: 33414 extracted. Category zero
File: 33415 extracted. Category zero
File: 33416 extracted. Category zero
File: 33417 extracted. Category zero
File: 33418 extracted. Category zero
File: 33419 extracted. Category zero
File: 33420 extracted. Category zero
File: 33421 extracted. Category zero
File: 33422 extracted. Category zero
File: 33423 extracted. Category zero
File: 33424 extracted. Category zero
File: 33425 extracted. Category zero
File: 33426 extracted. Category zero
F

File: 33623 extracted. Category zero
File: 33624 extracted. Category zero
File: 33625 extracted. Category zero
File: 33626 extracted. Category zero
File: 33627 extracted. Category zero
File: 33628 extracted. Category zero
File: 33629 extracted. Category zero
File: 33630 extracted. Category zero
File: 33631 extracted. Category zero
File: 33632 extracted. Category zero
File: 33633 extracted. Category zero
File: 33634 extracted. Category zero
File: 33635 extracted. Category zero
File: 33636 extracted. Category zero
File: 33637 extracted. Category zero
File: 33638 extracted. Category zero
File: 33639 extracted. Category zero
File: 33640 extracted. Category zero
File: 33641 extracted. Category zero
File: 33642 extracted. Category zero
File: 33643 extracted. Category zero
File: 33644 extracted. Category zero
File: 33645 extracted. Category zero
File: 33646 extracted. Category zero
File: 33647 extracted. Category zero
File: 33648 extracted. Category _background_noise_
File: 33649 extracted. C

In [94]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [95]:
y_train

array([['eight'],
       ['zero'],
       ['go'],
       ...,
       ['go'],
       ['cat'],
       ['no']], dtype='<U18')

In [ ]:
model = Sequential()
model.add(LSTM(units=128, dropout=0.01, recurrent_dropout=0.2, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(units=32, dropout=0.01, recurrent_dropout=0.2, return_sequences=False))
model.add(Dense(units= 256, activation='relu'))
model.add(Dense(units= 128, activation='relu'))
model.add(Dense(units= 64, activation='relu'))
model.add(Dense(units= 10, activation='softmax'))

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])